# Task for Today  

***

## Lyft/Uber Price Prediction  

Given *data about Lyft and Uber rides*, let's try to predict the **price** of a given ride.  
  
We will use a linear regression model to make our predictions.

# Getting Started

In [6]:
#importing libraries 
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

In [35]:
# reading csv /datset files 
rides_df = pd.read_csv('/cab_rides.csv')
weather_df = pd.read_csv('/weather.csv')

In [36]:
rides_df

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL
...,...,...,...,...,...,...,...,...,...,...
90109,0.56,Uber,1544677209238,Haymarket Square,North Station,NaN,1.0,0fdb8b85-20f6-4c06-8dc2-02e35fb2cde0,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
90110,0.56,Uber,1543772882500,Haymarket Square,North Station,7.0,1.0,4f4e647e-8307-44ce-a55d-8a010921a837,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV
90111,0.56,Uber,1545072913452,Haymarket Square,North Station,16.0,1.0,54a3a603-0e87-4e56-a08c-ab2dd5029177,6c84fd89-3f11-4782-9b50-97c468b19529,Black
90112,0.56,Uber,1543889884811,Haymarket Square,North Station,10.5,1.0,9bae879b-cc92-4a8e-a628-e7e0c3147cf9,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL


In [37]:
rides_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90114 entries, 0 to 90113
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   distance          90114 non-null  float64
 1   cab_type          90114 non-null  object 
 2   time_stamp        90114 non-null  int64  
 3   destination       90114 non-null  object 
 4   source            90114 non-null  object 
 5   price             83024 non-null  float64
 6   surge_multiplier  90114 non-null  float64
 7   id                90114 non-null  object 
 8   product_id        90113 non-null  object 
 9   name              90113 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 6.9+ MB


In [39]:
#printing weaather dataset 
weather_df

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.00,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.00,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.00,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.00,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.00,1012.14,0.1786,1545003901,0.75,11.49
...,...,...,...,...,...,...,...,...
6271,44.72,North Station,0.89,1000.69,NaN,1543819974,0.96,1.52
6272,44.85,Northeastern University,0.88,1000.71,NaN,1543819974,0.96,1.54
6273,44.82,South Station,0.89,1000.70,NaN,1543819974,0.96,1.54
6274,44.78,Theatre District,0.89,1000.70,NaN,1543819974,0.96,1.54


In [40]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        6276 non-null   float64
 1   location    6276 non-null   object 
 2   clouds      6276 non-null   float64
 3   pressure    6276 non-null   float64
 4   rain        894 non-null    float64
 5   time_stamp  6276 non-null   int64  
 6   humidity    6276 non-null   float64
 7   wind        6276 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 392.4+ KB


# Cleaning Ride Data

In [41]:
# printing rides data 
rides_df

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL
...,...,...,...,...,...,...,...,...,...,...
90109,0.56,Uber,1544677209238,Haymarket Square,North Station,NaN,1.0,0fdb8b85-20f6-4c06-8dc2-02e35fb2cde0,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
90110,0.56,Uber,1543772882500,Haymarket Square,North Station,7.0,1.0,4f4e647e-8307-44ce-a55d-8a010921a837,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV
90111,0.56,Uber,1545072913452,Haymarket Square,North Station,16.0,1.0,54a3a603-0e87-4e56-a08c-ab2dd5029177,6c84fd89-3f11-4782-9b50-97c468b19529,Black
90112,0.56,Uber,1543889884811,Haymarket Square,North Station,10.5,1.0,9bae879b-cc92-4a8e-a628-e7e0c3147cf9,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL


In [42]:
# Checking null values in rides dataset 
rides_df.isna().sum()

distance               0
cab_type               0
time_stamp             0
destination            0
source                 0
price               7090
surge_multiplier       0
id                     0
product_id             1
name                   1
dtype: int64

In [43]:
# dropping null values in the data set row wise
rides_df = rides_df.dropna(axis=0).reset_index(drop=True)

# Cleaning Weather Data

In [44]:
# printing weatther data 
weather_df

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.00,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.00,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.00,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.00,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.00,1012.14,0.1786,1545003901,0.75,11.49
...,...,...,...,...,...,...,...,...
6271,44.72,North Station,0.89,1000.69,NaN,1543819974,0.96,1.52
6272,44.85,Northeastern University,0.88,1000.71,NaN,1543819974,0.96,1.54
6273,44.82,South Station,0.89,1000.70,NaN,1543819974,0.96,1.54
6274,44.78,Theatre District,0.89,1000.70,NaN,1543819974,0.96,1.54


In [45]:
# checking null values in weather data 
weather_df.isna().sum()

temp             0
location         0
clouds           0
pressure         0
rain          5382
time_stamp       0
humidity         0
wind             0
dtype: int64

In [46]:
# filling null values in weather data 
weather_df = weather_df.fillna(0)

# Creating Average Weather DataFrame

In [47]:
weather_df

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.00,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.00,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.00,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.00,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.00,1012.14,0.1786,1545003901,0.75,11.49
...,...,...,...,...,...,...,...,...
6271,44.72,North Station,0.89,1000.69,0.0000,1543819974,0.96,1.52
6272,44.85,Northeastern University,0.88,1000.71,0.0000,1543819974,0.96,1.54
6273,44.82,South Station,0.89,1000.70,0.0000,1543819974,0.96,1.54
6274,44.78,Theatre District,0.89,1000.70,0.0000,1543819974,0.96,1.54


In [48]:
# Calculating the mean/Average value of each prperty location wise 
weather_df.groupby('location').mean()

,temp,clouds,pressure,rain,time_stamp,humidity,wind
location,,,,,,,
Back Bay,39.082122,0.678432,1008.447820,0.007925,1.543857e+09,0.764073,6.778528
Beacon Hill,39.047285,0.677801,1008.448356,0.008297,1.543857e+09,0.765048,6.810325
Boston University,39.047744,0.679235,1008.459254,0.007738,1.543857e+09,0.763786,6.692180
Fenway,38.964379,0.679866,1008.453289,0.007343,1.543857e+09,0.767266,6.711721
Financial District,39.410822,0.676730,1008.435793,0.008563,1.543857e+09,0.754837,6.860019
Haymarket Square,39.067897,0.676711,1008.445239,0.008660,1.543857e+09,0.764837,6.843193
North End,39.090841,0.676730,1008.441912,0.008644,1.543857e+09,0.764054,6.853117
North Station,39.035315,0.676998,1008.442811,0.008649,1.543857e+09,0.765545,6.835755
Northeastern University,38.975086,0.678317,1008.444168,0.007358,1.543857e+09,0.767648,6.749426


In [49]:
avg_weather_df = weather_df.groupby('location').mean().reset_index(drop=False)
avg_weather_df = avg_weather_df.drop('time_stamp', axis=1)
avg_weather_df

,location,temp,clouds,pressure,rain,humidity,wind
0,Back Bay,39.082122,0.678432,1008.447820,0.007925,0.764073,6.778528
1,Beacon Hill,39.047285,0.677801,1008.448356,0.008297,0.765048,6.810325
2,Boston University,39.047744,0.679235,1008.459254,0.007738,0.763786,6.692180
3,Fenway,38.964379,0.679866,1008.453289,0.007343,0.767266,6.711721
4,Financial District,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019
5,Haymarket Square,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193
6,North End,39.090841,0.676730,1008.441912,0.008644,0.764054,6.853117
7,North Station,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
8,Northeastern University,38.975086,0.678317,1008.444168,0.007358,0.767648,6.749426
9,South Station,39.394092,0.677495,1008.438031,0.008310,0.755468,6.848948


# Merging DataFrames

In [50]:
rides_df

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL
...,...,...,...,...,...,...,...,...,...,...
83018,1.63,Uber,1544837711202,West End,South Station,19.5,1.0,f1b75279-575e-4244-9c25-5b3601b96919,6c84fd89-3f11-4782-9b50-97c468b19529,Black
83019,1.63,Uber,1543704478517,West End,South Station,9.0,1.0,fe36a72b-8f32-44fb-9972-2b0e3e9be934,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV
83020,0.56,Uber,1543772882500,Haymarket Square,North Station,7.0,1.0,4f4e647e-8307-44ce-a55d-8a010921a837,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV
83021,0.56,Uber,1545072913452,Haymarket Square,North Station,16.0,1.0,54a3a603-0e87-4e56-a08c-ab2dd5029177,6c84fd89-3f11-4782-9b50-97c468b19529,Black


In [51]:
source_weather_df = avg_weather_df.rename(
    columns={
        'location': 'source',
        'temp': 'source_temp',
        'clouds': 'source_clouds',
        'pressure': 'source_pressure',
        'rain': 'source_rain',
        'humidity': 'source_humidity',
        'wind': 'source_wind'
    }
)

source_weather_df

,source,source_temp,source_clouds,source_pressure,source_rain,source_humidity,source_wind
0,Back Bay,39.082122,0.678432,1008.447820,0.007925,0.764073,6.778528
1,Beacon Hill,39.047285,0.677801,1008.448356,0.008297,0.765048,6.810325
2,Boston University,39.047744,0.679235,1008.459254,0.007738,0.763786,6.692180
3,Fenway,38.964379,0.679866,1008.453289,0.007343,0.767266,6.711721
4,Financial District,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019
5,Haymarket Square,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193
6,North End,39.090841,0.676730,1008.441912,0.008644,0.764054,6.853117
7,North Station,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
8,Northeastern University,38.975086,0.678317,1008.444168,0.007358,0.767648,6.749426
9,South Station,39.394092,0.677495,1008.438031,0.008310,0.755468,6.848948


In [52]:
# Merging both ride and weather data frame  
destination_weather_df = avg_weather_df.rename(
    columns={
        'location': 'destination',
        'temp': 'destination_temp',
        'clouds': 'destination_clouds',
        'pressure': 'destination_pressure',
        'rain': 'destination_rain',
        'humidity': 'destination_humidity',
        'wind': 'destination_wind'
    }
)

destination_weather_df

,destination,destination_temp,destination_clouds,destination_pressure,destination_rain,destination_humidity,destination_wind
0,Back Bay,39.082122,0.678432,1008.447820,0.007925,0.764073,6.778528
1,Beacon Hill,39.047285,0.677801,1008.448356,0.008297,0.765048,6.810325
2,Boston University,39.047744,0.679235,1008.459254,0.007738,0.763786,6.692180
3,Fenway,38.964379,0.679866,1008.453289,0.007343,0.767266,6.711721
4,Financial District,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019
5,Haymarket Square,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193
6,North End,39.090841,0.676730,1008.441912,0.008644,0.764054,6.853117
7,North Station,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
8,Northeastern University,38.975086,0.678317,1008.444168,0.007358,0.767648,6.749426
9,South Station,39.394092,0.677495,1008.438031,0.008310,0.755468,6.848948


In [53]:
data = rides_df\
    .merge(source_weather_df, on='source')\
    .merge(destination_weather_df, on='destination')

data

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,source_temp,source_clouds,source_pressure,source_rain,source_humidity,source_wind,destination_temp,destination_clouds,destination_pressure,destination_rain,destination_humidity,destination_wind
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193,39.035315,0.676998,1008.442811,0.008649,0.765545,6.835755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83018,1.25,Uber,1543209242307,Haymarket Square,Financial District,5.5,1.0,1c3563f2-ca30-45bd-abd6-81935465e902,997acbb5-e102-41e1-b155-9df7de0a73f2,UberPool,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193
83019,1.25,Uber,1545054612022,Haymarket Square,Financial District,11.5,1.0,42ef5a96-d408-4a23-9227-147f060bee08,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193
83020,1.25,Uber,1544752515825,Haymarket Square,Financial District,7.5,1.0,4925783f-8929-453b-a608-fb18891c9d9b,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193
83021,1.25,Uber,1543770481497,Haymarket Square,Financial District,15.0,1.0,dd6076ee-23f2-42ff-90a5-7b41677ca60c,6c84fd89-3f11-4782-9b50-97c468b19529,Black,39.410822,0.676730,1008.435793,0.008563,0.754837,6.860019,39.067897,0.676711,1008.445239,0.008660,0.764837,6.843193


# Preprocessing

In [54]:
# writing function for creating dummy variables with the help of one hot encoding 
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [55]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop id column
    df = df.drop('id', axis=1)
    
    # Binary encode cab_type column
    df['cab_type'] = df['cab_type'].replace({'Lyft': 0, 'Uber': 1})
    
    # One-hot encode remaining categorical columns
    for column, prefix in [('destination', "dest"), ('source', "src"), ('product_id', "pid"), ('name', "nm")]:
        df = onehot_encode(df, column=column, prefix=prefix)
    
    # Split df into X and y
    y = df['price']
    X = df.drop('price', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [56]:
# splitting our data into test and train 
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [57]:
X_train

,distance,cab_type,time_stamp,surge_multiplier,source_temp,source_clouds,source_pressure,source_rain,source_humidity,source_wind,destination_temp,destination_clouds,destination_pressure,destination_rain,destination_humidity,destination_wind,dest_Back Bay,dest_Beacon Hill,dest_Boston University,dest_Fenway,dest_Financial District,dest_Haymarket Square,dest_North End,dest_North Station,dest_Northeastern University,dest_South Station,dest_Theatre District,dest_West End,src_Back Bay,src_Beacon Hill,src_Boston University,src_Fenway,src_Financial District,src_Haymarket Square,src_North End,src_North Station,src_Northeastern University,src_South Station,src_Theatre District,src_West End,pid_55c66225-fbe7-4fd5-9072-eab1ece5e23e,pid_6c84fd89-3f11-4782-9b50-97c468b19529,pid_6d318bcc-22a3-4af6-bddd-b409bfce1546,pid_6f72dfc5-27f1-42e8-84db-ccc7a75f6969,pid_997acbb5-e102-41e1-b155-9df7de0a73f2,pid_9a0e7b09-b92b-4c41-9779-2ad22b4d779d,pid_lyft,pid_lyft_line,pid_lyft_lux,pid_lyft_luxsuv,pid_lyft_plus,pid_lyft_premier,nm_Black,nm_Black SUV,nm_Lux,nm_Lux Black,nm_Lux Black XL,nm_Lyft,nm_Lyft XL,nm_Shared,nm_UberPool,nm_UberX,nm_UberXL,nm_WAV
0,-0.272695,0.982668,-0.472924,-0.167528,0.001279,-1.067926,-0.527859,0.894469,0.015735,0.915584,-0.287116,0.004955,0.480562,0.197559,0.245026,0.159785,-0.300015,3.350705,-0.312617,-0.307751,-0.302461,-0.303003,-0.304456,-0.298991,-0.308421,-0.289986,-0.300900,-0.290544,-0.300764,-0.303713,-0.298957,-0.29377,-0.305299,-0.303274,3.368489,-0.297726,-0.299026,-0.299503,-0.309056,-0.30989,-0.306174,3.279097,-0.303477,-0.301002,-0.305029,-0.305602,-0.300083,-0.299230,-0.298752,-0.299333,-0.298957,-0.295391,3.279097,-0.303477,-0.295391,-0.298752,-0.299333,-0.300083,-0.298957,-0.299230,-0.305029,-0.306174,-0.301002,-0.305602
1,-1.527505,-1.017638,1.383728,2.321241,2.196795,-1.067926,-1.526850,0.724851,-2.170910,1.042936,2.124513,-0.303217,-1.176707,0.224436,-2.060522,0.855370,-0.300015,-0.298445,-0.312617,-0.307751,-0.302461,-0.303003,-0.304456,-0.298991,-0.308421,3.448441,-0.300900,-0.290544,-0.300764,-0.303713,-0.298957,-0.29377,3.275478,-0.303274,-0.296869,-0.297726,-0.299026,-0.299503,-0.309056,-0.30989,-0.306174,-0.304962,-0.303477,-0.301002,-0.305029,-0.305602,3.332413,-0.299230,-0.298752,-0.299333,-0.298957,-0.295391,-0.304962,-0.303477,-0.295391,-0.298752,-0.299333,3.332413,-0.298957,-0.299230,-0.305029,-0.306174,-0.301002,-0.305602
2,0.087076,0.982668,1.555817,-0.167528,-0.156152,-1.087658,0.015342,0.927909,0.201736,0.732494,-0.044863,0.640559,0.394629,-0.571204,0.010330,-0.412868,3.333170,-0.298445,-0.312617,-0.307751,-0.302461,-0.303003,-0.304456,-0.298991,-0.308421,-0.289986,-0.300900,-0.290544,-0.300764,-0.303713,-0.298957,-0.29377,-0.305299,3.297351,-0.296869,-0.297726,-0.299026,-0.299503,-0.309056,-0.30989,-0.306174,-0.304962,-0.303477,3.322241,-0.305029,-0.305602,-0.300083,-0.299230,-0.298752,-0.299333,-0.298957,-0.295391,-0.304962,-0.303477,-0.295391,-0.298752,-0.299333,-0.300083,-0.298957,-0.299230,-0.305029,-0.306174,3.322241,-0.305602
3,-0.834288,-1.017638,1.273494,-0.167528,-0.156152,-1.087658,0.015342,0.927909,0.201736,0.732494,-0.708334,-0.033567,-0.099482,0.419691,0.953718,0.591599,-0.300015,-0.298445,-0.312617,-0.307751,-0.302461,-0.303003,-0.304456,-0.298991,-0.308421,-0.289986,3.323367,-0.290544,-0.300764,-0.303713,-0.298957,-0.29377,-0.305299,3.297351,-0.296869,-0.297726,-0.299026,-0.299503,-0.309056,-0.30989,-0.306174,-0.304962,-0.303477,-0.301002,-0.305029,-0.305602,-0.300083,-0.299230,3.347255,-0.299333,-0.298957,-0.295391,-0.304962,-0.303477,-0.295391,3.347255,-0.299333,-0.300083,-0.298957,-0.299230,-0.305029,-0.306174,-0.301002,-0.305602
4,-1.044886,-1.017638,-1.089913,-0.167528,2.196795,-1.067926,-1.526850,0.724851,-2.170910,1.042936,-0.143785,-1.092908,-0.019688,0.947351,0.194406,0.751721,-0.300015,-0.298445,-0.312617,-0.307751,-0.302461,3.300297,-0.304456,-0.298991,-0.308421,-0.289986,-0.300900,-0.290544,-0.300764,-0.303713,-0.298957,-0.29377,3.275478,-0.303274,-0.296869,-0.297

In [58]:
y_train

21703    19.5
75102     9.0
28265    27.0
13485    16.5
82964    26.0
         ... 
21440    13.5
73349    16.5
50057     3.5
5192      9.0
77708     9.0
Name: price, Length: 58116, dtype: float64

# Training

In [60]:
# creating a model and fitting our traing data 
# checking accuracy of our model 
model = LinearRegression()
model.fit(X_train, y_train)

print("Test R^2 Score: {:.5f}".format(model.score(X_test, y_test)))


Test R^2 Score: 0.92708
